In [234]:
import pandas as pd
import numpy as np


In [235]:
df = pd.DataFrame([1, 2988.826, 3041.892, 3155.796, 2994.507], [2023, 2022, 2021, 2020, 2019])

In [236]:
df.rename(columns={0: 'Bauinvestitionen Kanton St. Gallen'}, inplace=True)
df.index.name = 'Date'

In [237]:
#interpolation of days
df.head()

,Bauinvestitionen Kanton St. Gallen
Date,
2023,1.000
2022,2988.826
2021,3041.892
2020,3155.796
2019,2994.507


In [238]:
# monthly_weights = {
#     "January": 0.07,  # Coldest month, lowest weight
#     "February": 0.07,  # Cold month, low weight
#     "March": 0.08,
#     "April": 0.08,
#     "May": 0.085,
#     "June": 0.100,  # Hot month, higher weight
#     "July": 0.110,  # Hottest month, highest weight
#     "August": 0.100,  # Hot month, high weight
#     "September": 0.085,
#     "October": 0.080,
#     "November": 0.07,
#     "December": 0.07  # Cold month, lowest weight
# }

# # Number of days per month (non-leap year)
# days_per_month = {
#     "January": 31, "February": 28, "March": 31, "April": 30,
#     "May": 31, "June": 30, "July": 31, "August": 31,
#     "September": 30, "October": 31, "November": 30, "December": 31
# }

# # Adjust the days for leap years
# days_per_month_leap = days_per_month.copy()
# days_per_month_leap["February"] = 29  # February has 29 days in leap years

# # Function to check if a year is a leap year
# def is_leap_year(year):
#     return (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)

# # Function to calculate daily weight for a given date
# def calculate_daily_weight(date):
#     month_name = date.strftime('%B')  # Get the full month name (e.g., 'Januar')
#     year = date.year
    
#     # Check if the year is a leap year
#     is_leap = is_leap_year(year)
#     days_in_month = days_per_month_leap if is_leap else days_per_month
    
#     # Get the number of days in the month
#     days = days_in_month[month_name]
    
#     # Calculate the daily weight for that month
#     monthly_weight = monthly_weights[month_name]
#     daily_weight = monthly_weight / days
    
#     return daily_weight

# # Interpolate daily values
# df.index = pd.to_datetime(df.index, format='%Y')
# # Resample the data to daily frequency and interpolate
# df_daily = df.resample('D').interpolate()

# # Forward fill the first of each month
# df_daily.loc[df_daily.index.is_month_start, 'Bauinvestitionen Kanton St. Gallen'] = df['Bauinvestitionen Kanton St. Gallen']
# df_daily['Bauinvestitionen Kanton St. Gallen'].fillna(method='ffill', inplace=True)

# # Apply the calculate_daily_weight function to the 'Date' index to get the daily weight
# df_daily['Daily Weight'] = df_daily.index.to_series().apply(calculate_daily_weight)

# # Ensure the daily values are the same within each month for every month
# for date, value in zip(df.index, df['Bauinvestitionen Kanton St. Gallen']):
#     df_daily.loc[df_daily.index.to_period('M') == date.to_period('M'), 'Bauinvestitionen Kanton St. Gallen'] = value

# print(df_daily.head(1000))  

# # Adjust the values based on the daily weights
# df_daily['Bauinvestitionen Kanton St. Gallen'] *= df_daily['Daily Weight']

# # Check for duplicates in df_daily
# duplicates = df_daily[df_daily.duplicated()]
# print("Duplicates in df_daily:")
# #print(duplicates)

# print(df_daily.head())

# # Calculate the total for each year
# yearly_totals = df_daily.resample('Y').sum()

# print(yearly_totals)

#interpolation gives us a faulty yearly total, as the daily values are not exactly the same for each day of the month (interpolation works by 
# taking the average of the two values, which is not the same as the original value splitted by every day of the month/year)     


In [239]:
df.head()

,Bauinvestitionen Kanton St. Gallen
Date,
2023,1.000
2022,2988.826
2021,3041.892
2020,3155.796
2019,2994.507


In [240]:
# Resample the data to daily frequency and fill with yearly value divided by the number of days
df.index = pd.to_datetime(df.index, format='%Y')
df_daily = df.resample('D').ffill()

print(df_daily.loc["2022"])


            Bauinvestitionen Kanton St. Gallen
Date                                          
2022-01-01                            2988.826
2022-01-02                            2988.826
2022-01-03                            2988.826
2022-01-04                            2988.826
2022-01-05                            2988.826
...                                        ...
2022-12-27                            2988.826
2022-12-28                            2988.826
2022-12-29                            2988.826
2022-12-30                            2988.826
2022-12-31                            2988.826

[365 rows x 1 columns]


In [241]:
#remove 2023 column 
df.drop(pd.Timestamp('2023-01-01'), inplace=True)
# Fill the daily values with the yearly value divided by the number of days in the year
for date, value in zip(df.index, df['Bauinvestitionen Kanton St. Gallen']):
    days_in_year = 366 if date.is_leap_year else 365
    daily_value = value / days_in_year
    df_daily.loc[df_daily.index.year == date.year, 'Bauinvestitionen Kanton St. Gallen'] = daily_value

print(df_daily.head(1000))

# Calculate and print the sum for each year
for year in df.index.year:
    yearly_sum = df_daily.loc[str(year), 'Bauinvestitionen Kanton St. Gallen'].sum()
    print(f"Sum of year {year}: {yearly_sum}")




            Bauinvestitionen Kanton St. Gallen
Date                                          
2019-01-01                            8.204129
2019-01-02                            8.204129
2019-01-03                            8.204129
2019-01-04                            8.204129
2019-01-05                            8.204129
...                                        ...
2021-09-22                            8.333951
2021-09-23                            8.333951
2021-09-24                            8.333951
2021-09-25                            8.333951
2021-09-26                            8.333951

[1000 rows x 1 columns]
Sum of year 2022: 2988.8259999999996
Sum of year 2021: 3041.8920000000003
Sum of year 2020: 3155.7960000000003
Sum of year 2019: 2994.5070000000005


In [244]:
# Calculate and print the sum for each year
for year in df.index.year:
    yearly_sum = df_daily.loc[str(year), 'Bauinvestitionen Kanton St. Gallen'].sum()
    print(f"Sum of year {year}: {yearly_sum}")
    # Define monthly weights
    monthly_weights = {
        "January": 0.07,  # Coldest month, lowest weight
        "February": 0.07,  # Cold month, low weight
        "March": 0.08,
        "April": 0.08,
        "May": 0.085,
        "June": 0.10,  # Hot month, higher weight
        "July": 0.11,  # Hottest month, highest weight
        "August": 0.10,  # Hot month, high weight
        "September": 0.085,
        "October": 0.08,
        "November": 0.07,
        "December": 0.07  # Cold month, lowest weight
    }

    # Function to calculate daily weight for a given date
    def calculate_daily_weight(date):
        month_name = date.strftime('%B')  # Get the full month name (e.g., 'January')
        days_in_month = days_per_month_leap if date.is_leap_year else days_per_month
        days = days_in_month[month_name]
        monthly_weight = monthly_weights[month_name]
        daily_weight = monthly_weight / days
        return daily_weight

    # Apply the calculate_daily_weight function to the 'Date' index to get the daily weight
    df_daily['Daily Weight'] = df_daily.index.to_series().apply(calculate_daily_weight)

    # Adjust the values based on the daily weights
    df_daily['Bauinvestitionen Kanton St. Gallen'] *= df_daily['Daily Weight']


Sum of year 2022: 1.746742175325881e-17
Sum of year 2021: 5.723866293609136e-20
Sum of year 2020: 1.9251453645577313e-22
Sum of year 2019: 6.059322745247603e-25
